In [2]:
#Importar Galerías#

#galerias de dataframes
import pandas as pd

#Navegar en web
from selenium import webdriver  # Permite navegar en la web
import re
import time 
import unidecode 
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from tqdm import tqdm
from selenium.webdriver.common.action_chains import ActionChains 

#Simulador de webdriver 
from webdriver_manager.chrome import ChromeDriverManager

#Warnings library
import warnings
warnings.filterwarnings('ignore')

In [3]:
# 1. Entrar a página web
driver_juntos = webdriver.Chrome( ChromeDriverManager().install() )

# 2. Dirección de juntos
url = 'http://www2.juntos.gob.pe/infojuntos/'

# 3. Acceder a la dirección de juntos con get
driver_juntos.get( url )

# 4. Acceder a la sección de VRAEM
# Se usa función find_element haciendo un seguimiento a la ruta directa
# Se usa ruta de referencia unica que es indexv.html
driver_juntos.find_element(By.XPATH,"//a[@href='indexv.html']").click()


In [4]:
all_tables = {}

#Lista de años:
anio = [2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]

In [5]:
for i in anio:   
    #Exportar información en formato excel:
    
    # 5. usar el ID 'canho' para seleccionar la lista desplegable de años
    driver_juntos.find_element(By.ID,"canho").click()
   
    # 6. Seleccionar el i-esimo año
    driver_juntos.find_element(By.XPATH, 
                                        "//option[@value={}]".format(f'"{i}"')).click()
    # 7. Tiempo de espera
    time.sleep(3)
     
    # 8. Dar click en alguna parte de la pantalla (para no seguir en boton de años), en este caso se da click en el titulo
    driver_juntos.find_element(By.ID, "titulodatos").click()
    
    # 9. Seleccionar boton de exportación 
    driver_juntos.find_element(By.XPATH,
        "//button[@class='btn btn-success bg-red dropdown-toggle']").click()
    
    # 10. Seleccionar opcion de 'exportar datos' que se describe como bimexp en el html
    driver_juntos.find_element(By.XPATH,"//*[@id='bimexp']").click()   ###SE DESCARGA EN FORMATO EXCEL##########
    
    #Construir tablas:
    
    # 11. Obtener la tabla que está dentro de la ruta  "/html/body/div[1].... " con 'get_attribute'
    table_html=driver_juntos.find_element(By.XPATH,
    "/html/body/div[1]/div/section[2]/div[2]/section[2]/div[1]/div[2]").get_attribute('innerHTML')
    # 12. Convertirlo a formato dataframe
    table = pd.read_html(table_html)
    table = table[0]
    # 13. Juntar todas las tablas de todos los años en diccionario
    all_tables[f"table_{i}"] = table
    
    ###Repetir proceso para los i-años de la muestra del 2014 al 2022

In [6]:
    #15. Extraer cada tabla (de cada año) del diccionario
a=all_tables[f"table_2014"]
    #16. Eliminar columnas que no son anuales
a=a.drop(['I BIM.', 'II BIM.', 'III BIM.', 'IV BIM.','V BIM.', 'VI BIM.'], axis=1)
a.rename(columns={'ANUAL': '2014', 'Unnamed: 0': 'ANUAL'}, inplace=True)
#a=a.set_index('ANUAL')
a=a.drop([6,7,9], axis=0)
a

,ANUAL,2014
0,Departamentos atendidos,5
1,Provincias atendidas,10
2,Distritos atendidos,51
3,Hogares afiliados,52022
4,Hogares abonados,46506
5,CCPP en CIAP con hogares afiliados,135
8,Transferencia S/.,55488149


### PARTE 2: CONSTRUIR TABLA - IMAGEN 2

In [7]:
all_tables2 = {}
for i in anio: 
    #15. Extraer cada tabla (de cada año) del diccionario
    a=all_tables[f"table_{i}"]
    #16. Eliminar columnas que no son anuales
    a=a.drop(['I BIM.', 'II BIM.', 'III BIM.', 'IV BIM.','V BIM.', 'VI BIM.'], axis=1)
    a.rename(columns={'ANUAL': '2014', 'Unnamed: 0': 'ANUAL'}, inplace=True)
    a=a.drop([6,7,9], axis=0)
    a=a.set_index('ANUAL')
    #18. Juntar información en nuevo diccionario
    all_tables2[f"tables_{i}"] = a.T

In [8]:
#20. Tabla final
total_data = pd.concat(all_tables2.values()).reset_index( drop = True )
total_data['year']=anio  #se agregan años
total_data

ANUAL,Departamentos atendidos,Provincias atendidas,Distritos atendidos,Hogares afiliados,Hogares abonados,CCPP en CIAP con hogares afiliados,Transferencia S/.,year
0,5,10,51,52022,46506,135,55488149,2014
1,5,10,59,50354,47431,141,58021852,2015
2,5,10,68,46487,40778,203,50809264,2016
3,5,10,69,42878,38896,208,48194288,2017
4,5,10,69,39132,36557,212,45530750,2018
5,5,10,69,39137,35864,178,42119689,2019
6,5,10,69,36385,36237,175,43859095,2020
7,5,10,69,35606,34299,251,41799198,2021


In [9]:
with open('pregunta2_2021.tex','w') as tf:
    tf.write(total_data.to_latex())